In [1]:
from mlflowrate.workflow import WorkFlow

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, IntegerType
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import lit, unix_timestamp
from pyspark.sql import functions

In [5]:
dummy1 = [("JJ", 1.0, 2, 3, 8, 5), ("Bizarre", 1.8, 3, 3, 5, None)]
df1 = spark.createDataFrame(dummy1, ["datetime", "a", "b", "c", "d", "e"])

dummy2 = [("John", 1.0, 2, 3, 4, 5), ("Snow", 1.3, None, 4, 5, 6), ("JJ", 1.0, 2, 3, 8, 5), ("Bizarre", 1.8, 3, 3, 5, 6)]
df2 = spark.createDataFrame(dummy2, ["datetime", "a", "b", "c", "d", "e"])

In [9]:
dfs = {"d1":df1, 
       "d2":df2}

flow = WorkFlow(dfs)

In [19]:
flow.integrate.merge_data(newname="df", first="d1", second="d2", axis=0)
flow.integrate.status("df")


df
~~~~~~~~~~~~~~~~~~
Organised (dataframe format == dictionary format): False
Number of samples in dataframe: 6
Number of date duplicates in dataframe: 2
Number of samples with null across columns in dictionary:
e  |  Samples 6  |  Nulls 1  |  Duplicates 2
b  |  Samples 6  |  Nulls 1  |  Duplicates 2
c  |  Samples 6  |  Nulls 0  |  Duplicates 2
a  |  Samples 6  |  Nulls 0  |  Duplicates 2
d  |  Samples 6  |  Nulls 0  |  Duplicates 2


In [23]:
flow.integrate.dfs["df"].show()

+--------+---+----+---+---+----+
|datetime|  a|   b|  c|  d|   e|
+--------+---+----+---+---+----+
|      JJ|1.0|   2|  3|  8|   5|
| Bizarre|1.8|   3|  3|  5|null|
|    John|1.0|   2|  3|  4|   5|
|    Snow|1.3|null|  4|  5|   6|
|      JJ|1.0|   2|  3|  8|   5|
| Bizarre|1.8|   3|  3|  5|   6|
+--------+---+----+---+---+----+

